In [5]:
import sys
sys.path.insert(0, '../data')

from risf.distance_functions import DegreeDivergenceDist, JaccardDist
from risf.distance import TestDistanceMixin, TrainDistanceMixin
from data_getter import get_glocalkd_dataset
import numpy as np
from pathlib import Path
import pickle
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
DATA_DIR = "../data/graph"
PRECOMPUTED_DISTANCES_PATH = Path("../precomputed_distances")
PRECOMPUTED_DISTANCES_PATH.mkdir(exist_ok=True)

dataset_names_SC = [ "NCI1", "PPAR-gamma"]
dataset_names_DB = ["DD", "p53"]
dataset_names_KC = ["IMDB-BINARY", "MMP",]
dataset_names_OS = ["REDDIT-BINARY", "HSE"]

distances = [JaccardDist(), DegreeDivergenceDist()] 

In [26]:
def precompute_graph_datasets(distances, dataset_name):
    for dataset_name in dataset_names_SC:
        data = get_glocalkd_dataset(DATA_DIR, dataset_name)
        for distance in distances:
            X_train = data["X_train"]
            train_distance = TrainDistanceMixin(distance)
            train_distance.precompute_distances(X_train, n_jobs=-4)
            pickle.dump(train_distance, open(PRECOMPUTED_DISTANCES_PATH / f"{dataset_name}_{distance.__class__.__name__}_train.pickle", 'wb'))

            X_test = data["X_test"]
            test_distance = TestDistanceMixin(distance, selected_objects = np.arange(X_train.shape[0]))
            test_distance.precompute_distances(X_train, X_test= X_test, n_jobs=-4)
            pickle.dump(test_distance, open(PRECOMPUTED_DISTANCES_PATH / f"{dataset_name}_{distance.__class__.__name__}_test.pickle", 'wb'))
        

In [7]:
precompute_graph_datasets(distances, dataset_names_SC)

In [ ]:
precompute_graph_datasets(distances, dataset_names_DB)

In [ ]:
precompute_graph_datasets(distances, dataset_names_KC)

In [ ]:
precompute_graph_datasets(distances, dataset_names_OS)